In [1]:
import numpy as np
import scipy
import scipy.linalg
import matplotlib.pyplot as plt

# Base class from which our solver inherits
from numerical_eqs.pde.sdole import SDOLEPDESolver

from utils import *


def as_interwoven( M ):
    return M.T.reshape(-1)

def as_stacked( M, bandWidth ):
    return M.reshape(-1, bandWidth).T

In [2]:

class AnelasticSolver(SDOLEPDESolver):
    ########### Boilerplate
    def __init__(self, rho_0, rho_0_p, rho_1, rho_1_p, g, err_fn = lambda x: np.linalg.norm(x, ord=np.inf)):
        
        # Set our constants
        self.rho_0 = rho_0
        self.rho_0_p = rho_0_p
        self.rho_1 = rho_1
        self.rho_1_p = rho_1_p
        self.g = g
        
        # The error function we'll use for step doubling purposes
        self.errfn = err_fn
        
    def setup(self, ):
        self.dx = (self.mesh[1:] - self.mesh[:-1])[0]
        
        # Value for p0 () must be imputed (self.u0[2] must be set implicitly)
        v0, v1, p0, p1, h = (self.u0[i] for i in range(self.u0.shape[0]))

        self.u0[2] = p1 + self.g * ( h * self.rho_0 + (1-h) * self.rho_1)
        # Make sure to mask out the zeros that shuld be there
        self.u0[-3:,-1] = 0
        
        print('u0')
        print(self.u0)
        
        
#         print(self.u0)
    def process(self, results):
        
        results['ys'] = np.asarray(results['ys'])
        results['time'] = np.asarray(results['time'])
        return results
    
    #################################
    def resid(self, U, dU, dt, dx):
        # Pull out stacked terms
#         print('U', U.T)
        W = U + dU
        V0 = W[0]
        V1 = W[1]
        P0 = W[2,:-1]
        P1 = W[3,:-1]
        H  = W[4,:-1]
        
        d_V0 = dU[0]
        d_V1 = dU[1]
        d_P0 = dU[2,:-1]
        d_P1 = dU[3,:-1]
        d_H  = dU[4,:-1]


        # The first residual for V0 is V0[0] and the last equation,
        # assuming N intervals is V0[N]; these tell us that V0 vanishes
        # at the ends of the interval at the advanced time.
        # The first and last residuals for V1 are similar.
        # The last residual for P0 is P0[N]; this unused value is to be zero.
        # This is the model for the last residual for P1 and H as well.
        ########
        Req1 = 0 * V0
        Req1[1:-1] = self.rho_0 * d_V0[1:-1] / dt  + (P0[1:] - P0[:-1]) / dx

        
        ########
        Req2 = 0 * V0
        Req2[1:-1] = self.rho_1 * d_V1[1:-1] / dt + (P1[1:] - P1[:-1]) / dx

        
        ########
        Req3 = 0 * V0
        Req3[:-1] = (self.rho_0_p * H * d_P0 / dt) + \
                (self.rho_0 * d_H / dt) + \
                (self.rho_0 * (V0[1:] - V0[:-1])/dx)
        
        ########
        Req4 = 0 * V0
        Req4[:-1] = (self.rho_1_p * (1-H) * d_P1 / dt) - \
                (self.rho_1 * d_H / dt) + \
                (self.rho_1 * (V1[1:] - V1[:-1])/dx)

        
        ########
        Req5 = 0 * V0
        Req5[:-1] = P1 - P0 + self.g * (H * self.rho_0 + (1-H) * self.rho_1)

        ########
        
        R = np.concatenate([ Req1, Req2, Req3, Req4, Req5 ], axis=0).reshape(5,-1)
        print('R')
        print(R.T)
        return R
    
    
    
    def step_raw(self, x, t, dt):
        U = x
        
        # Get our initial guess
        R_b = self.resid(U, 0 * U, dt, self.dx)
        
        print('R_b', R_b)
        
        R_b = as_interwoven(R_b)
        

        
        def a_func ( du ):
            du = as_stacked(du, U.shape[0])
            r = self.resid( U, du, dt, self.dx)
            return as_interwoven(r)
            
        A = banded_jacobian_approx(
            f = a_func,
            x0 = as_interwoven( 0 * U ),
            n_low = 3,
            n_high = 3,
            eps=1e-8,
        )
        # Verify that A was built properly
#         print('dx', self.dx, 1/self.dx)
#         print('dt', dt, 1/dt)
        print('A', A.shape)
        print(np.sign(A.T))
        
#         print('W', (U+dU).T)
        
        dU = scipy.linalg.solve_banded(
            (3,3),
            A,
            -1 * R_b,
        )
        
        return as_stacked( dU, U.shape[0] )



    def step(self, x, t, dt):
        '''Use Backward Euler
        '''
        print('Taking Step')
        U = x
        
        U_course = U + self.step_raw(U, t, dt)
        U_fine_mid = U + self.step_raw(U, t, dt/2.0)
        U_fine = U_fine_mid + self.step_raw(U_fine_mid, t+dt/2.0, dt/2.0)
        
        # We're not correcting for nonlinearity here
        U_new = U_fine
        
        U_err = self.errfn(U_fine - U_course)
        
        return U_new, U_err

In [3]:
# 1 + N
meshsize = 1 + 10
mesh = np.linspace(0, 5, meshsize, endpoint=True)
print('mesh', mesh)
def q(s):
    return np.where(
        s <= 0,
        0,
        np.where(
            s < 1,
            s**2 * (3-2*s),
            1
        )
    )
U0 = np.concatenate([
    
    # v0 component
    0 * mesh,
    
    # v1 component
    0 * mesh,
    
    # p0 component
    0 * mesh,
    
    # p1 component
    0 * mesh,
    
    # h component
    0.2 * q(4*mesh/5) + 0.4,
    
], axis=0).reshape(5,-1)


t0, t1 = (0, 4)

plot_points = np.linspace(t0, t1, 10)
# Points in time that will be explicitly plotted below
time_points = [0,2,4]


explicit_times = {
    'time points': time_points + plot_points.tolist(),
}



# These are useless in this iteration
boundaries = (
    {'type': 'neumann', 'f': lambda t: 0},
    {'type': 'neumann', 'f': lambda t: 0},
)


pde = AnelasticSolver(
    rho_0 = 1,
    rho_0_p = 1e-6,
    rho_1 = 1/2,
    rho_1_p = 0.5e-6,
    g = 1,
)
res = pde.solve(
    mesh = mesh,
    u0 = U0,
    t0 = t0,
    t1 = t1,
    # Add in boundaries
    boundaries = boundaries,
    
    explicit_times = explicit_times,
    # Show the progress bar
    progress = True,
    
    # Just disable step doubling
    time_controls = {
        'dtmin': 1e-3,
        'dtmax': 1,
        'tol': 1e-3,
    }
)


sol_y = res['ys'][:,1,:]
sol_t = res['time']

  0%|          | 0/4 [00:00<?, ?it/s]

mesh [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
u0
[[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.    ]
 [0.7    0.7352 0.7896 0.8    0.8    0.8    0.8    0.8    0.8    0.8
  0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.    ]
 [0.4    0.4704 0.5792 0.6    0.6    0.6    0.6    0.6    0.6    0.6
  0.    ]]
Taking Step
R
[[0.     0.     0.     0.     0.    ]
 [0.0704 0.     0.     0.     0.    ]
 [0.1088 0.     0.     0.     0.    ]
 [0.0208 0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]]
R_b [[0.     0.0704 0.1088 0.0208 0.     0.     0.     0.     0.     0.
  0.    ]
 [0.     0.     0.     0.     0.     0

LinAlgError: singular matrix

In [ ]:


k = 3
cols = 2
rows = int(np.ceil(k / cols))
fig, axs = plt.subplots(rows, cols, figsize=(7*cols,4*rows))
axs = np.asarray(axs).flatten()


j = np.zeros(sol_t.shape)
for t in plot_points:
    j = np.logical_or(j, sol_t == t)

# Find times that satisfy
times = np.nonzero( j )
# Plot this using the colorbar
cf = axs[0].contourf(
    sol_y[times, :][0,:,:]
)
fig.colorbar(cf, ax=axs[0])
axs[0].set_title('Visual representation of solution Ut')
axs[0].set_xlabel('mesh x')
axs[0].set_ylabel('Time')





j = np.zeros(sol_t.shape)
for t in time_points:
    j = np.logical_or(j, sol_t == t)

# Find times that satisfy
times = np.asarray(np.nonzero( j )).flatten()

for i, t in zip(times, time_points):
    axs[1].plot(
        mesh,
        sol_y[i,:],
        label='t={0:.2f}'.format(t)
    )

axs[1].set_title('U at t in {0}'.format(time_points))
axs[1].set_xlabel('mesh x')
axs[1].set_ylabel('Solution Ut')
axs[1].legend()
axs[1].grid()





for i in range(0, len(mesh), len(mesh)//5):
    axs[2].plot(
        sol_t,
        sol_y[:,i],
        label='x={0:.2f}'.format(mesh[i])
    )
axs[2].set_title('Found solution')
axs[2].set_xlabel('Time t')
axs[2].set_ylabel('Solution Ut')
axs[2].legend()
axs[2].grid()



axs[3].plot(
    sol_t[:-1],
    np.log(sol_t[1:] - sol_t[:-1]),
)
axs[3].set_title('SDOLE time step dt')
axs[3].set_xlabel('Time t')
axs[3].set_ylabel('dt')
axs[3].grid()

fig.tight_layout()
plt.show()

None